In [1]:
cd ..


/Users/samweiss/src/ibotta_uplift


In [27]:
import numpy as np
import pandas as pd

from dataset.data_simulation import get_simple_uplift_data
from ibotta_uplift.ibotta_uplift import IbottaUplift
from ggplot import *


In [28]:
y, x, t = get_simple_uplift_data(10000)

y = pd.DataFrame(y)
y.columns = ['revenue','cost', 'noise']
y['profit'] = y['revenue'] - y['cost']

In [ ]:
uplift_model = IbottaUplift()
param_grid = dict(num_nodes=[8], dropout=[.1, .5], activation=[
                          'relu'], num_layers=[1, 2], epochs=[25], batch_size=[30])

uplift_model.fit(x, y, t.reshape(-1,1), param_grid = param_grid, n_jobs = 1)

/Users/samweiss/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
erupt_curves, dists = uplift_model.get_erupt_curves()

In [ ]:
dists['weights_1'] = [np.float(x.split(',')[0]) for x in dists['weights']]
erupt_curves['weights_1'] = [np.float(x.split(',')[0]) for x in erupt_curves['weights']]
ggplot(aes(x='weights_1', y='mean', group = 'assignment', colour = 'assignment'), data=erupt_curves) +\
    geom_line()+\
    geom_point()+facet_wrap("response_var_names")

In [ ]:
ggplot(aes(x='weights_1', y='num_observations'), data=dists) +\
    geom_line()+\
    geom_point()+facet_wrap('tmt')

In [ ]:
uplift_model.calibrate()


In [ ]:
erupt_curves, dists = uplift_model.get_erupt_curves(calibrator = True)
dists['weights_1'] = [np.float(x.split(',')[0]) for x in dists['weights']]
erupt_curves['weights_1'] = [np.float(x.split(',')[0]) for x in erupt_curves['weights']]
ggplot(aes(x='weights_1', y='mean', group = 'assignment', colour = 'assignment'), data=erupt_curves) +\
    geom_line()+\
    geom_point()+facet_wrap("response_var_names")

In [21]:
test  = uplift_model.copy()
test.x.shape


(10000, 2)

In [26]:
uplift_model.save('/users/samweiss/temp')

FileNotFoundError: [Errno 2] No such file or directory: '/users/samweiss/temp/ibotta_uplift_class.pkl'

In [23]:
test = IbottaUplift()
test.load('/users/samweiss/')
test

In [24]:
test.x

array([[0.78060603, 0.4817593 ],
       [0.34065289, 0.15929436],
       [0.19596946, 0.16828371],
       ...,
       [0.33434548, 0.07010037],
       [0.88742743, 0.15125775],
       [0.41579423, 0.38199395]])

array([[0.23260872, 0.26564861],
       [0.90174874, 0.53831439],
       [0.15531573, 0.85931222],
       ...,
       [0.65635404, 0.4829386 ],
       [0.03263312, 0.65077151],
       [0.70547124, 0.9619202 ]])

In [ ]:
uplift_model.copy().model == uplift_model.model

In [ ]:
import copy
test_copy = copy.copy(uplift_model)


In [ ]:
class test_class(object):
    def fit(self, x):
        self.x = x
test = test_class()
test.fit(1)
test.x


import dill
dill.dump(test, file = open('/users/samweiss/downloads/test_load.pkl', "wb"))
test_load = dill.load(open('/users/samweiss/downloads/test_load.pkl', "rb"))
test_load.x

In [ ]:
import dill
dill.dump(test, file = open('/users/samweiss/downloads/test_load.pkl', "wb"))
test_load = dill.load(open('/users/samweiss/downloads/test_load.pkl', "rb"))
test_load.x